<a href="https://colab.research.google.com/github/gemma24/my_first_repo/blob/main/sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm

1. Get the ticker symbol form the user
2. Use the symbol to goto the profile page
3. Get the table
* for each row get the pay
* Use pay to calulate tax
* Add tax to table
4. Save as CSV
5. Save as SQLite

In [ ]:
# Usually run of Google Coolab, so install unique packages
!pip install kora -q

#load packages
from bs4 import BeautifulSoup
from kora.selenium import wd
import pandas as pd


In [ ]:
def build_url():
  ''' This function will build the URL of the profile page'''
  ticker = input("Please type in a ticker symbol ")
  url_pattern = 'https://au.finance.yahoo.com/quote/{}/profile?p={}'
  result = url_pattern.format(ticker, ticker)
  return result

In [ ]:
def get_record(row):
  cells = row.find_all('td')
  name = cells[0].text
  title =  cells[1].text
  pay = cells[2].text
  return {'name': name, 'title': title, 'pay': pay}


In [ ]:
def get_rows(soup):
  # Get the table!
  table = soup.find('table')

  # get to the row
  body = table.find('tbody')
  rows = body.find_all('tr')
  return rows

In [ ]:
def calculate_tax(income):
  if income <= 18200:
    tax_payable = 0
  elif 18200 < income <= 45000:
    tax_payable = 0.19 * (income - 18200)
  elif 45000 < income < 120000:
    tax_payable = 5092 + 0.32 * (income - 45000)
  elif 120001 < income < 180000:
    tax_payable = 29467 + 0.37 * (income - 120000)
  elif income > 180000:
    tax_payable = 51667 + 0.45 * (income - 180000)


  return tax_payable

In [ ]:
def get_salary(pay):  
  last_char = pay[-1]
  if last_char == 'M':
    pay_as_num = float(pay[0:-1])
    sal = pay_as_num * 1000000
  elif last_char == 'k':
    pay_as_num = float(pay[0:-1])
    sal = pay_as_num * 1000
  else: # Must be a N/A
    sal = 0
  return sal


In [ ]:
url = build_url()
wd.get(url)
soup = BeautifulSoup(wd.page_source,'html.parser')
rows = get_rows(soup)

# getting all the records
records = []
for row in rows:
  rec = get_record(row)
  records.append(rec)

# adding the tax colum
for rec in records:
  sal = get_salary(rec['pay'])
  rec['tax'] = calculate_tax(sal)

exec_df = pd.DataFrame.from_records(records)
exec_df

Please type in a ticker symbol nab.ax


,name,title,pay,tax
0,"Mr. Ross Maxwell McEwan CBE, B.B.S.","Group CEO, MD & Director",2.54M,1113667.0
1,"Mr. Gary Andrew Lennon BEC, F.C.A., BEc (Hons)...",Group Chief Financial Officer,1.13M,479167.0
2,Mr. Patrick F. Wright,Group Exec. of Technology & Enterprise Operations,1.72M,744667.0
3,Ms. Sharon J. Cook,Group Exec. of Legal & Commercial Services,912.98k,381508.0
4,Ms. Susan Ferrier,Group Exec. of People & Culture,929.27k,388838.5
5,Ms. Angela Mentis B.Bus,"Group Chief Digital, Data & Analytics Officer",1.62M,699667.0
6,Mr. Shaun Dooley,Chief Risk Officer,1.05M,443167.0
7,Mr. Andrew Irvine,Chief Exec. of Bus. Private Banking Division,333.27k,120638.5
8,Mr. Lesile D. Matheson,Group Chief Operating Officer,N/A,0.0
9,Ms. Sally Mihell,Head of Investor Relations,N/A,0.0
